# Machine Learning Assignment 2 - Classification Model Implementation

## STUDENT INFORMATION
* **BITS ID:** 2025AB05236
* **Name:** FOUZAN ASHRAF
* **Email:** 2025ab05236@wilp.bits-pilani.ac.in
* **Program:** M.Tech AI & ML (BITS Pilani)

In [9]:
#Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (accuracy_score, roc_auc_score, precision_score, 
                             recall_score, f1_score, matthews_corrcoef)

In [10]:
# Model Imports
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

# 1. Data Loading
Reading data from the parent directory

In [11]:
try:
    # Get the current working directory
    current_dir = os.getcwd()
    print(f"Current working directory: {current_dir}")
    
    # List files in the current directory to verify data.csv exists
    files = os.listdir(current_dir)
    print(f"Files in directory: {files}")
    
    # Attempt to read the file with full path
    file_path = os.path.join(current_dir, 'data.csv')
    df = pd.read_csv(file_path)
    print("File loaded successfully!")
except FileNotFoundError:
    print("Error: data.csv not found in the current directory.")
    print("Please check the file name and location.")

Current working directory: C:\Users\fouza\Documents\Learning\git-repos\BITS-MTech-Fouzan-ML-Assignment2
Files in directory: ['.git', '.ipynb_checkpoints', 'app.py', 'model', 'README.md', 'requirements.txt']
Error: data.csv not found in the current directory.
Please check the file name and location.
